In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from torch import cuda, bfloat16
import transformers
import os
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Need auth token for these
hf_auth = os.environ.get('hf_token')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/home/hb/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

## Table data

In [3]:
import json
from tablular_eval_util import combine_csv_files, split_dataframe, preprocess_data, run_llm_inference

directory = '/home/hb/dataset_bgp/bgp_tab_dataset_test'
combined_df = combine_csv_files(directory)
if 'anomaly_status' in combined_df.columns:
    combined_df = combined_df.drop(columns=['anomaly_status'])
    

# Split the DataFrame into smaller chunks
split_size = 20
data_list = split_dataframe(combined_df, split_size)

# Preprocess the data into the required format
formatted_data = [preprocess_data(chunk) for chunk in data_list]

formatted_data_file = 'llm_table_bgp_data_test_20.json'
with open(formatted_data_file, 'w') as f:
    json.dump(formatted_data, f, indent=4)

with open(formatted_data_file, 'r') as f:
    formatted_data = json.load(f)

output_results_file = 'llm_7B_bgp_data_with_outputs_20.json'
run_llm_inference(formatted_data, model, tokenizer, max_length=3050, output_results_file=output_results_file)

Processed 1/10
Processed 2/10
Processed 3/10
Processed 4/10
Processed 5/10
Processed 6/10
Processed 7/10
Processed 8/10
Processed 9/10
Processed 10/10
[{'instruction': 'The goal for this task is to determine if the data indicates an anomaly. The context, section, and table columns provide important information for identifying the correct anomaly type.', 'input_seg': '[TLE] The context is about BGP data analysis for detecting anomalies. The section is related to a specific time period of BGP monitoring. [TAB] col: | timestamp | asn | num_routes | num_new_routes | num_withdrawals | num_origin_changes | num_route_changes | max_path_length | avg_path_length | max_edit_distance | avg_edit_distance | num_announcements | num_unique_prefixes_announced | row 1: | 2022-03-28 07:00:00 | 8342 | 0 | 0 | 0 | 0 | 0 | 0 | 0.0 | 0 | 0.0 | 0 | 0 | [SEP] row 2: | 2022-03-28 07:05:00 | 8342 | 7 | 7 | 0 | 0 | 0 | 0 | 0.0 | 0 | 0.0 | 19 | 7 | [SEP] row 3: | 2022-03-28 07:10:00 | 8342 | 0 | 0 | 7 | 0 | 0 | 0

In [4]:
import pandas as pd
import re

llama7b_outputs = pd.read_json("/home/hb/LLM-research/LLM_testing/llm_7B_bgp_data_with_outputs_20.json")
llama7b_outputs = llama7b_outputs["output"]
llama7b_outputs.to_csv("llm_7B_bgp_data_with_outputs_20.csv")

timestamps = []

timestamp_pattern = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}')

for output in llama7b_outputs:
    matches = timestamp_pattern.findall(output)
    timestamps.extend(matches)

llama7b_outputs_timestamp = list(set(timestamps))

print(timestamps)

['2022-03-28 08:25:00', '2022-03-28 08:35:00', '2022-03-28 08:25:00', '2022-03-28 08:35:00', '2022-03-28 09:45:00', '2022-03-28 10:15:00', '2022-03-28 10:10:00', '2022-03-28 09:50:00', '2022-03-28 10:05:00', '2022-03-28 09:35:00', '2022-03-28 09:40:00', '2022-03-28 09:45:00', '2022-03-28 10:15:00', '2022-03-28 10:10:00', '2022-03-28 09:50:00', '2022-03-28 10:05:00', '2022-03-28 09:35:00', '2022-03-28 09:40:00', '2022-03-28 10:25:00', '2022-03-28 10:30:00', '2022-03-28 10:35:00', '2022-03-28 11:05:00', '2022-03-28 11:10:00', '2022-03-28 11:25:00', '2022-03-28 11:40:00', '2022-03-28 11:45:00', '2022-03-28 13:25:00', '2022-03-28 13:35:00', '2022-03-28 13:20:00', '2022-03-28 13:25:00', '2022-03-28 13:35:00', '2022-03-28 13:20:00', '2022-03-28 14:05:00', '2022-03-28 14:15:00', '2022-03-28 14:25:00', '2022-03-28 14:35:00', '2022-03-28 13:40:00', '2022-03-28 13:50:00', '2022-03-28 14:00:00', '2022-03-28 15:30:00', '2022-03-28 15:45:00', '2022-03-28 16:00:00', '2022-03-28 16:15:00', '2022-03-2

In [5]:
import pandas as pd
from tablular_eval_util import combine_csv_files

directory = '/home/hb/dataset_bgp/bgp_tab_dataset_test'
combined_df = combine_csv_files(directory)
combined_df = combined_df[['anomaly_status']]

# Filter the rows with "anomaly detected" in the anomaly_status
filtered_df = combined_df[combined_df['anomaly_status'].str.contains('anomaly detected', na=False)]
filtered_df.to_csv('/home/hb/dataset_bgp/bgp_tab_dataset_test/test_true_label.csv', index=False)

# Extract the date from the anomaly_status string
true_label = filtered_df['anomaly_status'].str.extract(r'anomaly detected at (\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})')

# Convert the dates to a list
true_label = true_label[0].tolist()

print(true_label)

['2022-03-28 10:25:00', '2022-03-28 10:30:00', '2022-03-28 10:35:00', '2022-03-28 10:40:00', '2022-03-28 10:55:00', '2022-03-28 11:00:00', '2022-03-28 11:35:00', '2022-03-28 11:40:00', '2022-03-28 11:45:00', '2022-03-28 13:25:00', '2022-03-28 13:30:00', '2022-03-28 13:35:00', '2022-03-28 14:00:00', '2022-03-28 14:05:00', '2022-03-28 14:15:00', '2022-03-28 14:20:00', '2022-03-28 14:25:00', '2022-03-28 15:55:00', '2022-03-28 16:00:00', '2022-03-28 16:05:00', '2019-05-08 15:05:00', '2019-05-08 15:10:00', '2019-05-08 15:15:00', '2019-05-08 15:30:00', '2017-12-12 04:40:00', '2017-12-12 04:45:00', '2017-12-12 04:50:00', '2017-12-12 07:05:00', '2017-12-12 07:10:00']


In [6]:
import pandas as pd
from tablular_eval_util import evaluate_llm_results

evaluation_result = evaluate_llm_results(true_anomalies=true_label, llm_results=llama7b_outputs_timestamp)
print("Evaluation Results:")
print(f"Precision: {evaluation_result['precision']:.2f}")
print(f"Recall: {evaluation_result['recall']:.2f}")
print(f"F1 Score: {evaluation_result['f1_score']:.2f}")
print(f"True Positives: {evaluation_result['true_positives']}")
print(f"False Positives: {evaluation_result['false_positives']}")
print(f"False Negatives: {evaluation_result['false_negatives']}")

Evaluation Results:
Precision: 0.21
Recall: 0.41
F1 Score: 0.28
True Positives: 12
False Positives: 46
False Negatives: 17


In [10]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = f""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1528)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 
The goal for this task is to identify the type of BGP anomaly based on the given data. The context, section, and table columns provide important information for identifying the correct anomaly type.. 
[TLE] The context is about BGP data analysis for detecting anomalies. The section is related to a specific time period of BGP monitoring. [TAB] col: | timestamp | asn | num_routes | num_new_routes | num_withdrawals | num_origin_changes | num_route_changes | max_path_length | avg_path_length | max_edit_distance | avg_edit_distance | num_announcements | num_unique_prefixes_announced | row 1: | 2024-06-10 12:00:00 | 12345 | 100 | 5 | 3 | 1 | 2 | 4 | 3.5 | 2 | 1.5 | 10 | 8 | [SEP] row 2: | 2024-06-10 12:05:00 | 12346 | 95 | 7 | 4 | 0 | 1 | 5 | 3.2 | 1 | 1.7 | 9 | 7 | [SEP] row 3: | 2024-06-10 12:10:00 | 12347 | 102 | 4 | 5 | 2 | 3 | 4.1 | 3 | 2.5 | 11 | 10 |
Based on the data provided, identify the type of BGP anomaly. The possible anomaly types are: route_flapping, outage, prefix_

In [ ]:
import pandas as pd
from tablular_eval_util import evaluate_llm_results

evaluation_result = evaluate_llm_results(true_anomalies=true_label, llm_results=llama7b_outputs_timestamp)
print("Evaluation Results:")
print(f"Precision: {evaluation_result['precision']:.2f}")
print(f"Recall: {evaluation_result['recall']:.2f}")
print(f"F1 Score: {evaluation_result['f1_score']:.2f}")
print(f"True Positives: {evaluation_result['true_positives']}")
print(f"False Positives: {evaluation_result['false_positives']}")
print(f"False Negatives: {evaluation_result['false_negatives']}")

## DDoS

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Create a Python script with the Scapy library to detect DDoS attacks from a PCAP file. The script should read the file, analyze traffic patterns by monitoring packets per second against a set threshold within defined time windows, and reset counters for each time window. It should also log any potential attacks showing the source IP and packet rate. Include a final check for the last time window at the end of the file."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=712)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Generate Python code to calculate and visualize 5G network performance KPIs: Total Network Capacity, Capacity per Area, Capacity per Point, Cost per Capacity, Cost per Area, and Surplus per Area. Load data from '5G_Infrastructure/demand_driven_postcode_data_results.csv'. Use keywords: 'capacity', 'cost', 'area', 'numpoints' to identify relevant columns."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=712)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Generate Python code to calculate and visualize 5G network performance KPIs: Total Network Capacity, Capacity per Area, Capacity per Point, Cost per Capacity, Cost per Area, and Surplus per Area. Load data from '5G_Infrastructure/demand_driven_postcode_data_results.csv'. Use keywords: 'capacity', 'cost', 'area', 'numpoints' to identify relevant columns. [/INST]  Sure! Here is some Python code that can be used to calculate and visualize 5G network performance KPIs based on the provided CSV file:
```
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data from the CSV file
df = pd.read_csv('5G_Infrastructure/demand_driven_postcode_data_results.csv')

# Identify the relevant columns
capacity = df['capacity']
cost = df['cost']
area = df['area']
numpoints = df['numpoints']

# Calculate the KPIs
total_network_capacity = capacity.sum()
capacity_per_area = capacity / area
capacity_per_point = capacity / numpoints
cost_per_capacity = cost / capacity


## Multiple Choice
`5G configuration analysis v1`:
Accuracy: 66%
Correct answers: 13 + 7 = 20
Incorrect answers: 10

`5G configuration analysis v2`:
Accuracy: 56%
Correct answers: 17
Incorrect answers: 13